In [2]:
import pandas as pd
import numpy as np

In [12]:
dfc = pd.read_csv('concat.csv', delimiter='|',decimal=".")
dfc.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13744\1245821509.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfc = pd.read_csv('concat.csv', delimiter='|',decimal=".")


,Date mutation,Nature mutation,Valeur fonciere,Type de voie,Code postal,Commune,Code departement,Code commune,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain
0,04/01/2018,Vente,90000.0,RUE,1380.0,SAINT-CYR-SUR-MENTHON,1,343,0,1.0,Maison,150.0,3.0,1,347.0
1,04/01/2018,Vente,67000.0,ALL,1000.0,BOURG-EN-BRESSE,1,53,1,2.0,Appartement,45.0,1.0,0,0.0
2,11/01/2018,Vente,76200.0,RUE,1000.0,BOURG-EN-BRESSE,1,53,2,2.0,Appartement,68.0,3.0,0,0.0
3,09/01/2018,Vente,90000.0,RUE,1380.0,BAGE-DOMMARTIN,1,25,0,1.0,Maison,120.0,3.0,1,2416.0
4,12/01/2018,Vente,130000.0,RUE,1160.0,VARAMBON,1,430,0,1.0,Maison,80.0,3.0,1,55.0


Calcul du prix au m2 par années en fonction du département

Filtre maison-appart

In [13]:
#Convertion de la colonne "Date mutation" au format date
dfc['Date mutation'] = pd.to_datetime(dfc['Date mutation'], format='%d/%m/%Y')
#Extraction de l'année de la colonne "date mutation" et créer une nouvelle colonne "année"
dfc['Annee']=dfc['Date mutation'].dt.year

In [14]:
dfc['Prix m²'] = dfc['Valeur fonciere'] / dfc['Surface reelle bati']

In [141]:
# Filtrer les lignes pour ne conserver que les locaux de type 'Appartement' et 'Maison
appart_maison = dfc[((dfc['Type local'] == 'Appartement') & (dfc['Surface reelle bati'] != 0)) | (dfc['Type local'] == 'Maison')]

# Calculer le prix au mètre carré pour ces locaux
appart_maison['Prix m2'] = appart_maison['Valeur fonciere'] / appart_maison['Surface reelle bati'].round(2)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18384\1801790639.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appart_maison['Prix m2'] = appart_maison['Valeur fonciere'] / appart_maison['Surface reelle bati'].round(2)


In [15]:
dfc['Code departement']=dfc['Code departement'].astype(str)
(dfc['Prix m²'] == np.inf).sum()

50415

In [16]:
dfc['Prix m²'] = dfc['Prix m²'].replace([np.inf, -np.inf], 0)

In [142]:
# On converti les départements en string pour éviter les doublons
appart_maison['Code departement'] = appart_maison['Code departement'].astype(str)


C:\Users\Admin\AppData\Local\Temp\ipykernel_18384\227622525.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appart_maison['Code departement'] = appart_maison['Code departement'].astype(str)


In [18]:
# On utilise pivot_table pour créer un tableau croisé avec le prix au mètre carré en fonction du département et de l'année
tableau_croise = pd.pivot_table(dfc, values=['Prix m²'], index=['Code departement'], columns=['Annee'], aggfunc='mean')

#Pour afficher le tableau entier, utiliser l'option:
pd.set_option('display.max_rows', None) 
tableau_croise

Prix m²                                          
Annee                    2018          2019          2020          2021
Code departement                                                       
1                 2067.052189   2158.966909   2244.647217   2374.889437
10                1303.968581   1356.994981   1332.118826   1690.886546
11                1464.919663   1501.749689   1563.443860   1545.950676
12                1218.644711   1216.685339   1236.847193   1263.998134
13                3058.258974   3082.905715   3234.129157   3893.557902
14                2192.802264   3073.591281   2376.656995   2608.430769
15                1052.873253   1120.104333   1102.230442   1142.151016
16                1060.143844   1230.410925   1186.389075   1247.412237
17                2267.584243   2384.017444   2512.757423   2698.884389
18                1113.765149   1530.806417   1142.073233   1128.468241
19                1134.722618   1159.232909   1202.525707   1294.586485
2                 1207.727313   1128.756632   1138.817005   1215.509040
21                1831.559457   1852.828582   1954.164215   2010.107751
22                1435.685178   1602.338707   1573.946111   1743.020607
23                 692.075002    779.496882    683.617763    700.294833
24                1234.481031   1261.456080   2953.278087   1418.246072
25                1683.984700   1776.551382   1798.787147   1926.431030
26                1793.138562   1767.454453   1910.561947   2049.381315
27                1675.140036   1724.392820   1923.482969   1935.436824
28                1661.699622   1688.806337   1726.710727   1830.525568
29                1460.834047   2555.109768   1634.882657   1858.408724
2A                3186.183527   3406.559543   3646.705953   5374.254024
2B                2761.537224   2620.175022   2697.143370   2964.918804
3                 1022.946738   1095.448215   1073.853831   1398.050142
30                2047.124441   3098.758841   2572.115904   2294.911264
31                2718.116205   2689.745298   2841.841957   2875.898079
32                1311.386373   1252.203351   1290.849864   1348.671594
33                3170.763462   3257.625667   3572.473423   3687.682905
34                2507.185585   2623.279285   2659.288194   2784.845068
35                2185.775839   2277.683525   2494.992911   2633.414431
36                 964.782353    901.447543    944.915693    933.957038
37                1869.627194   1966.804146   1988.902871   2157.719441
38                2194.241126   2328.062349   2365.262429   2620.765303
39                1304.643835   1320.049272   1454.603262  19463.871052
4                 1827.571573   1844.896000   2003.037336   1994.243944
40                2264.658029   2407.068341   2479.847798   2715.350577
41                1239.432647   1263.645992   1369.286844   1378.573782
42                1635.335105   1390.046019   1448.654622   1695.776830
43                1202.675383   1098.576196   1140.661099   1201.734363
44                2498.325072   2794.548086   2915.006132   3081.303807
45                1730.423995   1795.207107   1760.134302   1861.234211
46                1596.594300   1270.482642   1296.128024   1311.452355
47                1213.342846   1214.159367   1272.932929   1302.871182
48                1100.939439   1182.046301   1333.279834   1305.736592
49                1576.082748   1669.801770   1763.322593   1967.508295
5                 2213.436751   2186.410113   2239.048644   2384.001939
50                1606.349206   1558.424163   1651.394748   1796.338046
51                1788.725676   1787.377813   1883.368006   3857.990236
52                 879.144632    827.952211    820.610105    859.245896
53                1904.709921   1247.373655   1255.789339   2011.266590
54                1486.495586   1552.378835   1615.608641   1714.390092
55                 875.800589    828.974040    872.494180    924.305415
56                1936.455761   1995.839153   2136.915987   23

In [143]:
# On utilise pivot_table pour créer un tableau croisé avec le prix au mètre carré en fonction du département et de l'année
tableau_croise = pd.pivot_table(appart_maison, values=['Prix m2'], index=['Code departement'], columns=['Annee'], aggfunc='mean')

#Pour afficher le tableau entier, utiliser l'option:
#pd.set_option('display.max_rows', None) 
tableau_croise

Prix m2                                          
Annee                    2018          2019          2020          2021
Code departement                                                       
1                 2145.771358   2217.658084   2298.216821   2492.543695
10                1329.886510   1345.095819   1396.635322   1641.637547
11                1531.042295   1570.812932   1617.468047   1614.192426
12                1266.901390   1271.776388   1308.853049   1346.621798
13                3057.435817   3096.910457   3286.241133   4002.484601
14                2266.089329   3163.065393   2473.328376   2750.886691
15                1105.911232   1112.453423   1164.227160   1204.835308
16                1139.733427   1185.433765   1238.652513   1284.387190
17                2388.367688   2503.003184   2645.400827   2884.434788
18                1180.486980   1646.148951   1178.643872   1199.696991
19                1183.024058   1236.788898   1251.449158   1351.792915
2                 1248.773110   1155.259322   1191.988129   1272.437029
21                1874.741161   1901.463039   2016.017017   2124.566033
22                1461.168549   1594.537235   1621.175422   1816.122117
23                 729.639424    827.229101    724.797278    752.181238
24                1289.130940   1324.754392   3148.925545   1503.803165
25                1723.120697   1748.842744   1814.094138   1985.362175
26                1813.643545   1816.659022   1958.655757   2125.558667
27                1706.083285   1730.927272   1940.267925   1915.059006
28                1710.752179   1743.214288   1763.583940   1893.296421
29                1491.029409   2621.161142   1686.476303   1929.835039
2A                3233.188337   3433.829393   3537.359453   3893.910198
2B                2752.886633   2702.092582   2776.468915   2923.706194
3                 1067.662954   1146.965887   1107.522796   1225.464463
30                2088.110334   3065.555103   2641.811931   2326.953558
31                2684.034319   2746.384598   2897.754827   2987.538791
32                1380.736339   1340.597863   1380.237546   1434.769863
33                3236.795551   3314.391626   3517.416894   3793.586666
34                2538.955371   2644.384393   2676.350641   2854.268762
35                2269.093802   2342.627399   2551.378210   2681.649670
36                1030.313063    965.117333   1005.157522   1009.350064
37                1975.999677   2029.977922   2118.244863   2285.547229
38                2209.076159   2291.948419   2423.807268   2696.840030
39                1355.557286   1380.796378   1402.526664  21466.684539
4                 1883.827317   1916.172093   2089.593102   2074.823539
40                2313.671144   2451.328524   2523.086109   2792.853915
41                1307.700023   1313.506814   1421.720099   1487.594416
42                1699.591156   1424.969812   1494.581535   1724.203826
43                1247.559838   1144.224757   1195.938648   1270.825256
44                2584.214022   2882.439722   2964.422820   3225.396241
45                1785.994584   1781.969364   1810.041233   1965.479269
46                1400.702818   1348.238383   1382.171158   1426.937285
47                1276.183573   1264.258122   1322.074446   1379.609584
48                1157.044559   1221.140987   1387.621344   1379.925980
49                1635.621001   1720.681099   1852.680461   2053.166980
5                 2239.200697   2226.243943   2277.791930   2440.028569
50                1670.524486   1620.359027   1722.454755   1916.126435
51                1830.284420   1840.877305   1943.783748   4124.654505
52                 940.296978    895.179085    873.596078    908.644443
53                1235.742499   1245.159142   1287.668679   1457.493506
54                1558.324494   1599.793898   1690.108144   1794.066536
55                 932.698741    893.139145    939.856368    980.013381
56                2022.143489   2080.784188   2225.821603   24